<a href="https://colab.research.google.com/github/ataucuriaia/ESO-new-project/blob/main/ESO_new_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================================
# ESO Web Scraping + Enrichment Pipeline (v2.0)
# Enterprise Studio — Complete Pipeline with Expansion & People Integration
# 
# PHASES:
#   0) Setup & Data Load + Health Check
#   1) Phase 1: Org Expansion (discover new orgs from directories)
#   2) URL Normalization & Homepage Scraping (all orgs)
#   3) Homepage Signals Extraction
#   4) Org Capabilities Identification
#   5) Phase 2: People Extraction (all orgs) + Integration into org table
#   6) Single Master CSV Export
#
# INPUTS: CSV with columns "Org Name" and "Website URL"
# OUTPUTS: 
#   - Organization_Database_MASTER_v1_0.csv (single unified file with people integrated)
# ================================

# --- 1) Install + imports ---
# Handle pip install for both Colab and local environments
import subprocess
import sys

# Check and install missing packages
required_packages = {
    "pandas": "pandas",
    "requests": "requests",
    "beautifulsoup4": "bs4",
    "lxml": "lxml",
    "tqdm": "tqdm"
}

missing_packages = []
for package_name, import_name in required_packages.items():
    try:
        __import__(import_name)
    except ImportError:
        missing_packages.append(package_name)

if missing_packages:
    print(f"Installing missing packages: {', '.join(missing_packages)}...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + missing_packages)
        print("✓ Packages installed successfully")
        print("⚠ NOTE: If imports still fail, restart the kernel (Kernel → Restart)")
    except Exception as e:
        print(f"❌ Error installing packages: {e}")
        print(f"Please install manually: pip install {' '.join(missing_packages)}")
        raise

# Now import all packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

print("✓ All required packages imported successfully")

import re
import time
import json
import os
from urllib.parse import urlparse, urljoin

# --- 2) Load your existing database ---
INPUT_PATH = "Organization Database 1f24e34e337d8027b500d2a10b1ceaa7.csv"

try:
    df = pd.read_csv(INPUT_PATH)
except FileNotFoundError:
    raise FileNotFoundError(
        f"CSV file not found: {INPUT_PATH}\n"
        f"Please ensure the file is in the current directory: {os.getcwd()}"
    )

print(f"Loaded CSV: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Columns: {list(df.columns)}")

# Basic checks (matches your structure)
REQUIRED_COLS = ["Org Name", "Website URL"]
missing = [c for c in REQUIRED_COLS if c not in df.columns]
if missing:
    raise ValueError(
        f"Missing required columns: {missing}\n"
        f"Found columns: {list(df.columns)}\n"
        f"Please ensure your CSV has 'Org Name' and 'Website URL' columns."
    )

print("✓ Required columns present")

# --- 3) Helpers: URL cleaning + safe request ---
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )
}

def normalize_url(url: str) -> str:
    """Normalize website URL. Adds scheme if missing, strips whitespace."""
    if not isinstance(url, str) or not url.strip():
        return ""
    u = url.strip()
    # Common cleanup
    u = u.replace(" ", "")
    # If user typed "www.example.com" without scheme
    if u.startswith("www."):
        u = "https://" + u
    # If scheme missing but domain present
    if not re.match(r"^https?://", u) and "." in u:
        u = "https://" + u
    return u

def get_domain(url: str) -> str:
    try:
        return urlparse(url).netloc.lower()
    except Exception:
        return ""

def safe_get(url: str, timeout=20, max_retries=2, backoff=1.5):
    """
    HTTP GET with retries and improved error handling.
    Returns (final_url, html_text, error_msg) tuple.
    Handles rate limiting (429), content-type issues, and encoding problems.
    """
    if not url:
        return "", "", "Empty URL"
    
    last_err = None
    for attempt in range(max_retries + 1):
        try:
            r = requests.get(url, headers=HEADERS, timeout=timeout, allow_redirects=True)
            
            # Handle rate limiting with exponential backoff
            if r.status_code == 429:
                retry_after = int(r.headers.get("Retry-After", backoff ** attempt))
                if attempt < max_retries:
                    time.sleep(retry_after)
                    continue
                return r.url, "", f"HTTP 429 (rate limited) after {max_retries + 1} attempts"
            
            # Handle other HTTP errors
            if r.status_code in (403, 500, 502, 503, 504):
                if attempt < max_retries:
                    time.sleep(backoff ** attempt)
                    continue
                return r.url, "", f"HTTP {r.status_code}"
            
            # Only process successful responses (200-299)
            if not (200 <= r.status_code < 300):
                return r.url, "", f"HTTP {r.status_code}"
            
            # Check content type more flexibly
            content_type = (r.headers.get("Content-Type") or "").lower()
            # Some sites don't set Content-Type properly, so check if we got HTML-like content
            if content_type and "text/html" not in content_type:
                # Allow if content-type is missing but content looks like HTML
                if not content_type or ("text" not in content_type and "application" not in content_type):
                    # Try to detect HTML by checking first few bytes
                    try:
                        if r.text[:100].strip().startswith("<"):
                            pass  # Looks like HTML, proceed
                        else:
                            return r.url, "", f"Non-HTML content type: {content_type}"
                    except Exception:
                        return r.url, "", f"Non-HTML content type: {content_type}"
            
            # Handle encoding issues
            try:
                r.encoding = r.apparent_encoding if r.apparent_encoding else 'utf-8'
                html_text = r.text
            except UnicodeDecodeError as e:
                return r.url, "", f"Encoding error: {str(e)}"
            
            return r.url, html_text, ""
            
        except requests.exceptions.Timeout:
            last_err = f"Timeout after {timeout}s"
            if attempt < max_retries:
                time.sleep(backoff ** attempt)
                continue
        except requests.exceptions.ConnectionError as e:
            last_err = f"Connection error: {str(e)}"
            if attempt < max_retries:
                time.sleep(backoff ** attempt)
                continue
        except requests.exceptions.RequestException as e:
            last_err = f"Request error: {str(e)}"
            if attempt < max_retries:
                time.sleep(backoff ** attempt)
                continue
        except Exception as e:
            last_err = f"Unexpected error: {str(e)}"
            if attempt < max_retries:
                time.sleep(backoff ** attempt)
                continue
    
    return "", "", last_err or "Unknown error"

# --- 3.5) Health Check (after helpers defined, before expansion) ---
print("\n" + "="*60)
print("HEALTH CHECK (Original Database)")
print("="*60)

print(f"\n1. DataFrame Shape: {df.shape[0]} rows × {df.shape[1]} columns")

print(f"\n2. Required Columns Check:")
for col in REQUIRED_COLS:
    present = col in df.columns
    status = "✓" if present else "✗"
    print(f"   {status} '{col}': {'Present' if present else 'MISSING'}")

print(f"\n3. URL Validation:")
url_col = "Website URL"
if url_col in df.columns:
    total_urls = len(df)
    missing_urls = df[url_col].isna().sum() + (df[url_col].astype(str).str.strip() == "").sum()
    
    print(f"   Total rows: {total_urls}")
    print(f"   Missing/empty URLs: {missing_urls} ({100*missing_urls/total_urls:.1f}%)")
    print(f"   Valid URLs: {total_urls - missing_urls} ({100*(total_urls-missing_urls)/total_urls:.1f}%)")
    
    # Sample normalized URLs
    print(f"\n4. Sample Normalized URLs (first 10 non-empty):")
    sample_urls = df[df[url_col].notna() & (df[url_col].astype(str).str.strip() != "")][url_col].head(10)
    for i, raw_url in enumerate(sample_urls, 1):
        normalized = normalize_url(str(raw_url))
        print(f"   {i:2d}. {raw_url[:50]:50s} → {normalized[:60]}")
else:
    print(f"   ✗ '{url_col}' column not found!")

print("\n" + "="*60 + "\n")

# --- 3.6) Helper function for text cleaning (needed for expansion) ---
def clean_text(s: str) -> str:
    """Clean and normalize text."""
    s = re.sub(r"\s+", " ", s or "").strip()
    return s

# Rate limiting constant (used throughout)
RATE_LIMIT_SECONDS = 1.0  # be polite; tune later

# ================================
# PHASE 1: ORGANIZATION EXPANSION MODULE
# Discover and add new organizations from candidate sources
# ================================

# --- 3.7) Expansion Configuration ---
# Search API Configuration
# Auto-discovery will be enabled if API keys are found in environment variables
# Priority: Google CSE > SerpAPI > Bing
# To manually set provider: export SEARCH_PROVIDER=google_cse|serpapi|bing|none

# API Keys (read from environment variables only - NEVER hardcode)
GOOGLE_CSE_API_KEY = os.getenv("GOOGLE_CSE_API_KEY", "")
GOOGLE_CSE_ENGINE_ID = os.getenv("GOOGLE_CSE_ENGINE_ID", "")
SERPAPI_KEY = os.getenv("SERPAPI_KEY", "")
BING_SEARCH_KEY = os.getenv("BING_SEARCH_KEY", "")

# Debug: Show what keys are detected (without exposing actual keys)
print("\n" + "="*60)
print("="*60)
print("SEARCH API CONFIGURATION CHECK")
print("="*60)
print("="*60)
print(f"GOOGLE_CSE_API_KEY: {'✓ Set' if GOOGLE_CSE_API_KEY else '✗ Not set'} ({'***' + GOOGLE_CSE_API_KEY[-4:] if GOOGLE_CSE_API_KEY and len(GOOGLE_CSE_API_KEY) > 4 else 'N/A'})")
print(f"GOOGLE_CSE_ENGINE_ID: {'✓ Set' if GOOGLE_CSE_ENGINE_ID else '✗ Not set'}")
print(f"SERPAPI_KEY: {'✓ Set' if SERPAPI_KEY else '✗ Not set'} ({'***' + SERPAPI_KEY[-4:] if SERPAPI_KEY and len(SERPAPI_KEY) > 4 else 'N/A'})")
print(f"BING_SEARCH_KEY: {'✓ Set' if BING_SEARCH_KEY else '✗ Not set'} ({'***' + BING_SEARCH_KEY[-4:] if BING_SEARCH_KEY and len(BING_SEARCH_KEY) > 4 else 'N/A'})")

# Auto-detect provider if not explicitly set
SEARCH_PROVIDER_MANUAL = os.getenv("SEARCH_PROVIDER", "").lower()
if SEARCH_PROVIDER_MANUAL and SEARCH_PROVIDER_MANUAL != "none":
    SEARCH_PROVIDER = SEARCH_PROVIDER_MANUAL
    print(f"\nSEARCH_PROVIDER explicitly set to: {SEARCH_PROVIDER}")
    if SEARCH_PROVIDER == "serpapi" and not SERPAPI_KEY:
        print("  ⚠ WARNING: SEARCH_PROVIDER=serpapi but SERPAPI_KEY is not set!")
        print("  Please set SERPAPI_KEY environment variable or restart kernel after setting it.")
elif GOOGLE_CSE_API_KEY and GOOGLE_CSE_ENGINE_ID:
    SEARCH_PROVIDER = "google_cse"
    print("\n✓ Auto-discovery enabled: Google Custom Search (keys found in environment)")
elif SERPAPI_KEY:
    SEARCH_PROVIDER = "serpapi"
    print("\n✓ Auto-discovery enabled: SerpAPI (key found in environment)")
elif BING_SEARCH_KEY:
    SEARCH_PROVIDER = "bing"
    print("\n✓ Auto-discovery enabled: Bing Web Search (key found in environment)")
else:
    SEARCH_PROVIDER = "none"
    print("\nℹ Auto-discovery disabled: No API keys found in environment variables")
    print("  To enable, set one of:")
    print("    - GOOGLE_CSE_API_KEY + GOOGLE_CSE_ENGINE_ID")
    print("    - SERPAPI_KEY")
    print("    - BING_SEARCH_KEY")
    print("\n  NOTE: If you set environment variables, you may need to:")
    print("    1. Restart the Jupyter kernel (Kernel → Restart)")
    print("    2. Or set them in this notebook cell before running")
print("="*60 + "\n")

# ============================================
# OPTIONAL: Set API Key Directly in Notebook
# ============================================
# If environment variables aren't working, you can set the key here directly.
# WARNING: This will expose your API key in the notebook file.
# Only use this for testing, then remove it before committing to git.
# ============================================

# Uncomment and set your SerpAPI key here if environment variable isn't working:
SERPAPI_KEY = "07c20e0d88e60090f893e86c5b9da5ca0554041bfb237bec28322a6be183f2dc"
if SERPAPI_KEY and SERPAPI_KEY != "your_serpapi_key_here":
    print(f"✓ Using SerpAPI key from notebook (last 4 chars: ...{SERPAPI_KEY[-4:]})")
    SEARCH_PROVIDER = "serpapi"
    # Re-check configuration after setting key
    if SERPAPI_KEY:
        print("✓ Auto-discovery enabled: SerpAPI (key set in notebook)")

# ============================================

# Manual candidates (optional - can still provide manually)
CANDIDATE_ORGS = []  # Example: [{"name": "Org Name", "url": "https://example.com"}, ...]

# Seed directory URLs (optional)
SEED_DIRECTORY_URLS = []  # Example: ["https://directory.example.com/orgs", ...]

# Discovery limits
MAX_NEW_ORGS_PER_CAPABILITY = 25
MAX_TOTAL_NEW_ORGS = 100

def normalize_org_name(name: str) -> str:
    """Normalize org name for deduplication."""
    if not isinstance(name, str):
        return ""
    # Lowercase, strip, remove common suffixes
    normalized = name.lower().strip()
    normalized = re.sub(r'\s+', ' ', normalized)
    # Remove common legal suffixes
    normalized = re.sub(r'\b(inc|llc|corp|ltd|foundation|foundation|org|nonprofit)\b\.?$', '', normalized)
    return normalized.strip()

def search_orgs_via_api(capability: str, max_results: int = 20) -> list:
    """
    Search for organizations using configured search API.
    Returns list of candidate dicts with: name, url, title, snippet, query, matched_keywords, confidence_score
    """
    candidates = []
    
    if SEARCH_PROVIDER == "none" or not any([GOOGLE_CSE_API_KEY, SERPAPI_KEY, BING_SEARCH_KEY]):
        return candidates
    
    # Generate search queries for this capability
    queries = generate_search_queries(capability)
    
    for i, query in enumerate(queries, 1):
        try:
            if SEARCH_PROVIDER == "google_cse" and GOOGLE_CSE_API_KEY and GOOGLE_CSE_ENGINE_ID:
                results = search_google_cse(query, max_results=10)
            elif SEARCH_PROVIDER == "serpapi" and SERPAPI_KEY:
                results = search_serpapi(query, max_results=10)
            elif SEARCH_PROVIDER == "bing" and BING_SEARCH_KEY:
                results = search_bing(query, max_results=10)
            else:
                continue
            
            if len(results) > 0:
                print(f"      Query {i}/{len(queries)}: Found {len(results)} results")
            
            for result in results:
                # Quick classification
                confidence, matched_keywords = classify_candidate_confidence(
                    result.get("title", ""), 
                    result.get("snippet", ""), 
                    result.get("url", ""),
                    capability
                )
                
                if confidence > 0.3:  # High-confidence threshold
                    candidates.append({
                        "name": extract_org_name_from_title(result.get("title", "")),
                        "url": result.get("url", ""),
                        "title": result.get("title", ""),
                        "snippet": result.get("snippet", "")[:200],
                        "query": query,
                        "matched_keywords": "; ".join(matched_keywords),
                        "confidence_score": confidence,
                        "capability_bucket": capability
                    })
            
            time.sleep(0.5)  # Rate limit between queries
            
        except Exception as e:
            print(f"   Warning: Search error for query '{query}': {e}")
            continue
    
    return candidates

def generate_search_queries(capability: str) -> list:
    """Generate high-yield search queries for a capability."""
    queries = []
    
    if capability == "Regulatory / FDA":
        queries = [
            "FDA regulatory consulting nonprofit accelerator Virginia",
            "IND regulatory support organization",
            "FDA pathway guidance nonprofit",
            "regulatory affairs consulting startup support",
            "FDA submission support program"
        ]
    elif capability == "Clinical & Translational Support":
        queries = [
            "clinical translation support program Virginia",
            "translational research commercialization support",
            "clinical trials support innovation hub",
            "CRO support startup accelerator",
            "clinical development support nonprofit"
        ]
    elif capability == "IP / Legal / Licensing":
        queries = [
            "technology transfer licensing office Virginia",
            "startup IP legal clinic university",
            "patent support entrepreneurship nonprofit",
            "intellectual property support accelerator",
            "technology licensing support program"
        ]
    elif capability == "Manufacturing / GMP / Scale-Up":
        queries = [
            "GMP manufacturing support program Virginia",
            "scale-up manufacturing accelerator",
            "prototype to manufacturing support center",
            "manufacturing scale-up support nonprofit",
            "GMP compliance support startup"
        ]
    
    return queries[:5]  # Limit to 5 queries per capability

def classify_candidate_confidence(title: str, snippet: str, url: str, capability: str) -> tuple:
    """
    Quick classification of candidate confidence.
    Returns (confidence_score, matched_keywords_list).
    """
    combined_text = f"{title} {snippet}".lower()
    matched_keywords = []
    score = 0.0
    
    # Capability-specific keywords
    capability_keywords = {
        "Regulatory / FDA": ["fda", "regulatory", "regulatory affairs", "ind", "ide", "510k", "regulatory consulting"],
        "Clinical & Translational Support": ["clinical", "translational", "cro", "clinical trial", "clinical research", "translational research"],
        "IP / Legal / Licensing": ["ip", "intellectual property", "patent", "licensing", "technology transfer", "patent support"],
        "Manufacturing / GMP / Scale-Up": ["manufacturing", "gmp", "scale-up", "scaling", "production", "manufacturing support"]
    }
    
    keywords = capability_keywords.get(capability, [])
    for keyword in keywords:
        if keyword in combined_text:
            matched_keywords.append(keyword)
            score += 1.0
    
    # Organization indicators (positive)
    org_indicators = ["organization", "nonprofit", "foundation", "institute", "center", "program", 
                     "accelerator", "hub", "network", "alliance", "association"]
    for indicator in org_indicators:
        if indicator in combined_text:
            score += 0.5
    
    # Negative indicators (reduce confidence)
    negative_indicators = ["job", "careers", "hiring", "blog", "news", "article", "pdf", "download"]
    for neg in negative_indicators:
        if neg in combined_text:
            score -= 0.5
    
    # URL quality check
    if url:
        url_lower = url.lower()
        # Prefer official-looking domains
        if any(domain in url_lower for domain in [".org", ".edu", ".gov", ".com"]):
            if not any(bad in url_lower for bad in ["/blog/", "/news/", "/article/", "/pdf", ".pdf"]):
                score += 0.5
    
    # Normalize confidence (0-1 scale)
    confidence = min(1.0, max(0.0, score / 5.0))
    
    return confidence, matched_keywords

def extract_org_name_from_title(title: str) -> str:
    """Extract organization name from search result title."""
    if not title:
        return ""
    
    # Remove common prefixes/suffixes
    title = title.strip()
    # Remove "|", "-", "—" and everything after
    for sep in ["|", "-", "—", "::"]:
        if sep in title:
            title = title.split(sep)[0].strip()
    
    # Remove common suffixes
    title = re.sub(r'\s*-\s*(Home|Official|Website).*$', '', title, flags=re.I)
    
    return title[:100]  # Limit length

def search_google_cse(query: str, max_results: int = 10) -> list:
    """Search using Google Custom Search JSON API."""
    if not GOOGLE_CSE_API_KEY or not GOOGLE_CSE_ENGINE_ID:
        return []
    
    try:
        url = "https://www.googleapis.com/customsearch/v1"
        params = {
            "key": GOOGLE_CSE_API_KEY,
            "cx": GOOGLE_CSE_ENGINE_ID,
            "q": query,
            "num": min(max_results, 10)  # Google CSE max is 10 per request
        }
        
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            results = []
            for item in data.get("items", [])[:max_results]:
                results.append({
                    "title": item.get("title", ""),
                    "url": item.get("link", ""),
                    "snippet": item.get("snippet", "")
                })
            return results
    except Exception as e:
        print(f"   Google CSE error: {e}")
    
    return []

def search_serpapi(query: str, max_results: int = 10) -> list:
    """Search using SerpAPI."""
    if not SERPAPI_KEY:
        return []
    
    try:
        url = "https://serpapi.com/search"
        params = {
            "api_key": SERPAPI_KEY,
            "engine": "google",
            "q": query,
            "num": min(max_results, 10)
        }
        
        response = requests.get(url, params=params, timeout=15)
        if response.status_code == 200:
            data = response.json()
            results = []
            organic_results = data.get("organic_results", [])
            if not organic_results:
                # Check for API errors in response
                if "error" in data:
                    print(f"      ⚠ SerpAPI error: {data.get('error', 'Unknown error')}")
                return []
            
            for item in organic_results[:max_results]:
                results.append({
                    "title": item.get("title", ""),
                    "url": item.get("link", ""),
                    "snippet": item.get("snippet", "")
                })
            return results
        else:
            print(f"      ⚠ SerpAPI HTTP {response.status_code}: {response.text[:100]}")
            return []
    except requests.exceptions.Timeout:
        print(f"      ⚠ SerpAPI timeout for query: {query[:50]}")
        return []
    except Exception as e:
        print(f"      ⚠ SerpAPI error: {str(e)}")
        return []

def search_bing(query: str, max_results: int = 10) -> list:
    """Search using Bing Web Search API."""
    if not BING_SEARCH_KEY:
        return []
    
    try:
        url = "https://api.bing.microsoft.com/v7.0/search"
        headers = {"Ocp-Apim-Subscription-Key": BING_SEARCH_KEY}
        params = {
            "q": query,
            "count": min(max_results, 50),
            "responseFilter": "Webpages"
        }
        
        response = requests.get(url, headers=headers, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            results = []
            for item in data.get("webPages", {}).get("value", [])[:max_results]:
                results.append({
                    "title": item.get("name", ""),
                    "url": item.get("url", ""),
                    "snippet": item.get("snippet", "")
                })
            return results
    except Exception as e:
        print(f"   Bing API error: {e}")
    
    return []

def discover_orgs_from_directory(url: str, max_orgs: int = 50) -> list:
    """
    Discover candidate organizations from a directory page.
    Returns list of dicts with name and url.
    """
    discovered = []
    try:
        final_url, html, error_msg = safe_get(url)
        if not html or error_msg:
            return discovered
        
        soup = BeautifulSoup(html, "lxml")
        
        # Look for common patterns: links with org names, list items, etc.
        # Pattern 1: Links that look like org names (heuristic)
        for a in soup.find_all("a", href=True):
            text = clean_text(a.get_text())
            href = a.get("href", "")
            
            # Heuristic: org names are usually 2-6 words, start with capital
            words = text.split()
            if 2 <= len(words) <= 6 and text[0].isupper() and len(text) > 5:
                # Make URL absolute
                abs_url = urljoin(url, href)
                if abs_url.startswith("http"):
                    discovered.append({
                        "name": text,
                        "url": abs_url
                    })
                    if len(discovered) >= max_orgs:
                        break
        
    except Exception as e:
        print(f"   Warning: Error discovering orgs from {url}: {e}")
    
    return discovered

def deduplicate_new_orgs(existing_df: pd.DataFrame, candidate_orgs: list) -> tuple:
    """
    Deduplicate candidate orgs against existing database.
    Returns (new_orgs_list, duplicates_log).
    """
    if not candidate_orgs:
        return [], []
    
    # Build lookup sets from existing data
    existing_domains = set()
    existing_names_normalized = set()
    
    for idx, row in existing_df.iterrows():
        url = str(row.get("Website URL", ""))
        name = str(row.get("Org Name", ""))
        
        if url:
            domain = get_domain(normalize_url(url))
            if domain:
                existing_domains.add(domain)
        
        if name:
            existing_names_normalized.add(normalize_org_name(name))
    
    # Check candidates
    new_orgs = []
    duplicates_log = []
    
    for candidate in candidate_orgs:
        name = str(candidate.get("name", "")).strip()
        url = str(candidate.get("url", "")).strip()
        
        if not name or not url:
            continue
        
        # Normalize for comparison
        normalized_name = normalize_org_name(name)
        normalized_url = normalize_url(url)
        domain = get_domain(normalized_url)
        
        # Check for duplicates
        is_duplicate = False
        reason = ""
        
        if domain and domain in existing_domains:
            is_duplicate = True
            reason = f"Domain match: {domain}"
        elif normalized_name and normalized_name in existing_names_normalized:
            is_duplicate = True
            reason = f"Name match: {normalized_name}"
        
        if is_duplicate:
            duplicates_log.append({"name": name, "url": url, "reason": reason})
        else:
            new_org = {
                "Org Name": name,
                "Website URL": normalized_url,
                "Organization Type": "",
                "Long Name / Description": "",
                "Resources": "",
                "Industry ": "",
                "Track Record": "",
                "Charlottesville?": "",
                "Virginia?": ""
            }
            # Preserve capability_bucket if present (for tracking)
            if "capability_bucket" in candidate:
                new_org["capability_bucket"] = candidate["capability_bucket"]
            new_orgs.append(new_org)
    
    return new_orgs, duplicates_log

# --- 3.8) Backlog Management ---
BACKLOG_FILE = "expansion_backlog.csv"

def load_backlog() -> pd.DataFrame:
    """Load existing backlog file or create empty one."""
    if os.path.exists(BACKLOG_FILE):
        try:
            return pd.read_csv(BACKLOG_FILE)
        except Exception:
            pass
    
    # Create empty backlog
    return pd.DataFrame(columns=[
        "discovered_at", "capability_bucket", "org_name_guess", "url", "title", 
        "snippet", "query", "matched_keywords", "confidence_score", "status"
    ])

def save_backlog(backlog_df: pd.DataFrame):
    """Save backlog to CSV."""
    backlog_df.to_csv(BACKLOG_FILE, index=False)

def add_to_backlog(candidates: list, backlog_df: pd.DataFrame) -> pd.DataFrame:
    """Add new candidates to backlog, deduplicating by URL."""
    if not candidates:
        return backlog_df
    
    # Get existing URLs from backlog
    existing_urls = set(backlog_df["url"].astype(str).str.lower()) if "url" in backlog_df.columns else set()
    
    new_rows = []
    for candidate in candidates:
        url_lower = candidate.get("url", "").lower()
        if url_lower and url_lower not in existing_urls:
            new_rows.append({
                "discovered_at": time.strftime("%Y-%m-%d %H:%M:%S"),
                "capability_bucket": candidate.get("capability_bucket", ""),
                "org_name_guess": candidate.get("name", ""),
                "url": candidate.get("url", ""),
                "title": candidate.get("title", ""),
                "snippet": candidate.get("snippet", ""),
                "query": candidate.get("query", ""),
                "matched_keywords": candidate.get("matched_keywords", ""),
                "confidence_score": candidate.get("confidence_score", 0.0),
                "status": "queued"
            })
            existing_urls.add(url_lower)
    
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        backlog_df = pd.concat([backlog_df, new_df], ignore_index=True)
    
    return backlog_df

# ================================
# EXECUTE EXPANSION - THIS MUST RUN BEFORE SCRAPING
# ================================
print("\n" + "="*60)
print("="*60)
print("PHASE 1: ORGANIZATION EXPANSION")
print("="*60)
print("="*60)

original_count = len(df)
print(f"\nOriginal organizations: {original_count}")
print("Starting auto-discovery process...\n")

# Initialize variables
duplicates_log = []
all_candidates = []
discovery_log = []

# Load backlog
backlog_df = load_backlog()
print(f"Loaded backlog: {len(backlog_df)} candidates queued")

# Auto-discovery via search API
if SEARCH_PROVIDER != "none":
    print(f"\nAuto-discovery enabled (Provider: {SEARCH_PROVIDER})")
    
    if SEARCH_PROVIDER == "google_cse" and not (GOOGLE_CSE_API_KEY and GOOGLE_CSE_ENGINE_ID):
        print("  ⚠ Google CSE API key or Engine ID not found in environment variables")
        print("  Set GOOGLE_CSE_API_KEY and GOOGLE_CSE_ENGINE_ID to enable")
    elif SEARCH_PROVIDER == "serpapi" and not SERPAPI_KEY:
        print("  ⚠ SerpAPI key not found in environment variable")
        print("  Set SERPAPI_KEY to enable")
    elif SEARCH_PROVIDER == "bing" and not BING_SEARCH_KEY:
        print("  ⚠ Bing Search API key not found in environment variable")
        print("  Set BING_SEARCH_KEY to enable")
    else:
        # Search for each capability
        capabilities = ["Regulatory / FDA", "Clinical & Translational Support", 
                       "IP / Legal / Licensing", "Manufacturing / GMP / Scale-Up"]
        
        for capability in capabilities:
            print(f"\n  Searching for: {capability}")
            print(f"    Executing search queries... (this may take 10-30 seconds)")
            search_candidates = search_orgs_via_api(capability, max_results=20)
            print(f"    ✓ Search complete: Found {len(search_candidates)} candidates from search")
            
            if len(search_candidates) == 0:
                print(f"    ⚠ No candidates found. Check search queries or API response.")
            
            # Add to backlog
            backlog_df = add_to_backlog(search_candidates, backlog_df)
            
            # Add high-confidence candidates to processing queue
            high_confidence = [c for c in search_candidates if c.get("confidence_score", 0) > 0.5]
            all_candidates.extend(high_confidence)
            
            if len(high_confidence) > 0:
                print(f"    ✓ {len(high_confidence)} high-confidence candidates added to processing queue")
            
            discovery_log.append({
                "capability": capability,
                "results_pulled": len(search_candidates),
                "candidates": len(search_candidates),
                "high_confidence": len(high_confidence),
                "added": 0,  # Will update after deduplication
                "queued": len(search_candidates) - len(high_confidence)
            })
else:
    print("\nAuto-discovery disabled (SEARCH_PROVIDER='none')")
    print("  To enable, set environment variable SEARCH_PROVIDER to: google_cse, serpapi, or bing")
    print("  And set corresponding API key environment variables")

# Discover from directories
if SEED_DIRECTORY_URLS:
    print(f"\nDiscovering from {len(SEED_DIRECTORY_URLS)} directory URLs...")
    for directory_url in SEED_DIRECTORY_URLS:
        print(f"  Directory: {directory_url}")
        discovered = discover_orgs_from_directory(directory_url)
        print(f"    Found {len(discovered)} candidate orgs")
        all_candidates.extend(discovered)
        time.sleep(RATE_LIMIT_SECONDS)

# Add manual candidates
if CANDIDATE_ORGS:
    print(f"\nAdding {len(CANDIDATE_ORGS)} manual candidates...")
    all_candidates.extend(CANDIDATE_ORGS)

# Deduplicate and prioritize candidates
if all_candidates:
    print(f"\n{'='*60}")
    print(f"PROCESSING {len(all_candidates)} TOTAL CANDIDATES")
    print(f"{'='*60}")
    print(f"Prioritizing and filtering candidates...")
    
    # Prioritize by confidence score
    all_candidates_sorted = sorted(all_candidates, key=lambda x: x.get("confidence_score", 0), reverse=True)
    
    # Group by capability and limit per capability
    candidates_by_capability = {}
    for candidate in all_candidates_sorted:
        capability = candidate.get("capability_bucket", "unknown")
        if capability not in candidates_by_capability:
            candidates_by_capability[capability] = []
        if len(candidates_by_capability[capability]) < MAX_NEW_ORGS_PER_CAPABILITY:
            candidates_by_capability[capability].append(candidate)
    
    # Flatten back to list (limited per capability)
    prioritized_candidates = []
    for capability, candidates in candidates_by_capability.items():
        prioritized_candidates.extend(candidates)
        print(f"  {capability}: {len(candidates)} candidates selected (max {MAX_NEW_ORGS_PER_CAPABILITY} per capability)")
        # Update discovery log
        for log_entry in discovery_log:
            if log_entry.get("capability") == capability:
                log_entry["high_confidence"] = len(candidates)
    
    # Add remaining to backlog
    remaining_candidates = all_candidates_sorted[len(prioritized_candidates):]
    if remaining_candidates:
        backlog_df = add_to_backlog(remaining_candidates, backlog_df)
        print(f"  Added {len(remaining_candidates)} additional candidates to backlog")
    
    # Limit total new orgs
    if len(prioritized_candidates) > MAX_TOTAL_NEW_ORGS:
        prioritized_candidates = prioritized_candidates[:MAX_TOTAL_NEW_ORGS]
        print(f"  Limited to {MAX_TOTAL_NEW_ORGS} total new orgs (max limit)")
    
    print(f"\nDeduplicating against existing database...")
    # Deduplicate against existing database
    new_orgs, duplicates_log = deduplicate_new_orgs(df, prioritized_candidates)
    print(f"\n✓ Deduplication complete:")
    print(f"  Candidates processed: {len(prioritized_candidates)}")
    print(f"  New orgs to add: {len(new_orgs)}")
    print(f"  Duplicates skipped: {len(duplicates_log)}")
    
    # Update backlog status for added orgs
    if "url" in backlog_df.columns:
        added_urls = {org.get("Website URL", "").lower() for org in new_orgs}
        backlog_df.loc[backlog_df["url"].str.lower().isin(added_urls), "status"] = "added"
        
        duplicate_urls = {dup.get("url", "").lower() for dup in duplicates_log}
        backlog_df.loc[backlog_df["url"].str.lower().isin(duplicate_urls), "status"] = "skipped_duplicate"
    
    # Update discovery log with final counts
    for log_entry in discovery_log:
        capability = log_entry.get("capability")
        added_count = sum(1 for org in new_orgs if org.get("capability_bucket") == capability)
        log_entry["added"] = added_count
    
    # Create DataFrame for new orgs
    if new_orgs:
        df_new = pd.DataFrame(new_orgs)
        # Ensure all original columns exist
        for col in df.columns:
            if col not in df_new.columns:
                df_new[col] = ""
        
        # Combine: original + new
        df_org_all = pd.concat([df, df_new], ignore_index=True)
        print(f"\n{'='*60}")
        print(f"✓ EXPANSION COMPLETE: Unified org database created")
        print(f"{'='*60}")
        print(f"  Original orgs: {original_count}")
        print(f"  New orgs added: {len(new_orgs)}")
        print(f"  Total orgs to scrape: {len(df_org_all)}")
        print(f"{'='*60}\n")
    else:
        df_org_all = df.copy()
        print(f"\n{'='*60}")
        print(f"✓ EXPANSION COMPLETE: No new orgs to add")
        print(f"{'='*60}")
        print(f"  Using original database: {len(df_org_all)} orgs")
        print(f"{'='*60}\n")
else:
    df_org_all = df.copy()
    print(f"\n{'='*60}")
    print(f"✓ EXPANSION COMPLETE: No candidate orgs provided")
    print(f"{'='*60}")
    print(f"  Using original database: {len(df_org_all)} orgs")
    print(f"  (No new orgs discovered - auto-discovery may be disabled or found no candidates)")
    print(f"{'='*60}\n")

# CRITICAL: Ensure df_org_all is always defined before scraping
if 'df_org_all' not in locals():
    print("⚠ ERROR: df_org_all not defined! This should never happen.")
    df_org_all = df.copy()
    original_count = len(df)

# Save backlog
save_backlog(backlog_df)
print(f"✓ Saved backlog: {BACKLOG_FILE} ({len(backlog_df)} total candidates)")

# Save dedupe log if any duplicates
if duplicates_log:
    dedupe_df = pd.DataFrame(duplicates_log)
    dedupe_df.to_csv("dedupe_log.csv", index=False)
    print(f"✓ Saved deduplication log: dedupe_log.csv ({len(duplicates_log)} rows)")

# Save discovery log
if discovery_log:
    discovery_df = pd.DataFrame(discovery_log)
    discovery_df.to_csv("expansion_discovery_log.csv", index=False)
    print(f"✓ Saved discovery log: expansion_discovery_log.csv")
    
    # Print discovery summary
    print(f"\nDiscovery Summary:")
    for entry in discovery_log:
        print(f"  {entry['capability']}: {entry['results_pulled']} results, "
              f"{entry['high_confidence']} high-confidence, {entry['added']} added, "
              f"{entry['queued']} queued")

print("\n" + "="*60 + "\n")

# --- 4) HTML parsing: extract useful fields for your ESO DB ---
EMAIL_RE = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")

# clean_text is already defined above (needed for expansion module)

def extract_page_signals(base_url: str, html: str) -> dict:
    """
    Extract lightweight, high-signal fields from home page HTML.
    (You can extend this later: team page scraping, keyword tagging, etc.)
    Handles parsing errors gracefully.
    """
    try:
        soup = BeautifulSoup(html, "lxml")
    except Exception as e:
        # Fallback to html.parser if lxml fails
        try:
            soup = BeautifulSoup(html, "html.parser")
        except Exception as e2:
            raise ValueError(f"Failed to parse HTML: {str(e)}; fallback also failed: {str(e2)}")

    # Title - handle missing title gracefully
    try:
        title = clean_text(soup.title.get_text()) if soup.title else ""
    except Exception:
        title = ""

    # Meta description
    meta_desc = ""
    tag = soup.find("meta", attrs={"name": re.compile("^description$", re.I)})
    if tag and tag.get("content"):
        meta_desc = clean_text(tag["content"])

    # H1
    h1 = ""
    h1_tag = soup.find("h1")
    if h1_tag:
        h1 = clean_text(h1_tag.get_text())

    # Social links (common)
    socials = {"linkedin": "", "twitter_x": "", "youtube": "", "facebook": "", "instagram": ""}
    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        if "linkedin.com" in href and not socials["linkedin"]:
            socials["linkedin"] = href
        if ("twitter.com" in href or "x.com" in href) and not socials["twitter_x"]:
            socials["twitter_x"] = href
        if "youtube.com" in href and not socials["youtube"]:
            socials["youtube"] = href
        if "facebook.com" in href and not socials["facebook"]:
            socials["facebook"] = href
        if "instagram.com" in href and not socials["instagram"]:
            socials["instagram"] = href

    # Find contact/about/team page candidates (just links, not crawling yet)
    link_candidates = {"contact_url": "", "about_url": "", "team_url": ""}
    for a in soup.find_all("a", href=True):
        text = (a.get_text() or "").lower().strip()
        href = a["href"].strip()

        # Make absolute if relative
        abs_url = urljoin(base_url, href)

        if not link_candidates["contact_url"] and ("contact" in text or "contact" in href.lower()):
            link_candidates["contact_url"] = abs_url
        if not link_candidates["about_url"] and ("about" in text or "about" in href.lower() or "who we are" in text):
            link_candidates["about_url"] = abs_url
        if not link_candidates["team_url"] and (
            "team" in text or "our team" in text or "leadership" in text
            or "team" in href.lower() or "leadership" in href.lower()
        ):
            link_candidates["team_url"] = abs_url

    # Emails found on page
    emails = sorted(set(EMAIL_RE.findall(soup.get_text(" "))))
    emails = emails[:5]  # keep short

    # A short text snippet (useful for later tagging/classification)
    # Keep it lightweight: take first N chars from visible text
    try:
        page_text = clean_text(soup.get_text(" "))
        snippet = page_text[:600]
    except Exception:
        snippet = ""

    return {
        "site_title": title,
        "meta_description": meta_desc,
        "h1": h1,
        "text_snippet": snippet,
        "emails_found": "; ".join(emails),
        "contact_url_guess": link_candidates["contact_url"],
        "about_url_guess": link_candidates["about_url"],
        "team_url_guess": link_candidates["team_url"],
        "linkedin_url": socials["linkedin"],
        "twitter_x_url": socials["twitter_x"],
        "youtube_url": socials["youtube"],
        "facebook_url": socials["facebook"],
        "instagram_url": socials["instagram"],
    }

# ================================
# PHASE 2: URL NORMALIZATION & HOMEPAGE SCRAPING
# Scrape all organizations (existing + newly discovered)
# ================================

print("\n" + "="*60)
print("PHASE 2: HOMEPAGE SCRAPING")
print("="*60)
print(f"Starting to scrape {len(df_org_all)} organizations...")
print(f"(This includes {original_count} original + {len(df_org_all) - original_count} newly discovered orgs)")
print("="*60 + "\n")

# --- 5) Main loop: scrape each row (rate-limited) ---
# RATE_LIMIT_SECONDS is already defined above

enriched_rows = []
for idx, row in tqdm(df_org_all.iterrows(), total=len(df_org_all), desc="Scraping websites"):
    org = row.get("Org Name", "")
    raw_url = row.get("Website URL", "")
    url = normalize_url(raw_url)

    out = {
        "Org Name": org,
        "Website URL": raw_url,
        "website_normalized": url,
        "website_domain": get_domain(url),
        "final_url": "",
        "http_ok": False,
        "scrape_error": "",
    }

    if not url:
        out["scrape_error"] = "Missing URL"
        enriched_rows.append(out)
        continue

    final_url, html, error_msg = safe_get(url)
    
    if error_msg:
        out["scrape_error"] = error_msg
        enriched_rows.append(out)
        time.sleep(RATE_LIMIT_SECONDS)
        continue
    
    if not final_url:
        out["scrape_error"] = "Request failed (no URL returned)"
        enriched_rows.append(out)
        time.sleep(RATE_LIMIT_SECONDS)
        continue

    out["final_url"] = final_url

    if not html:
        out["scrape_error"] = "Non-HTML response or empty HTML"
        enriched_rows.append(out)
        time.sleep(RATE_LIMIT_SECONDS)
        continue

    try:
        signals = extract_page_signals(final_url, html)
        out.update(signals)
        # Store HTML for later extraction (truncated to save memory)
        out["homepage_html_stored"] = html[:50000] if html else ""  # Store first 50KB
        out["http_ok"] = True
    except Exception as e:
        out["scrape_error"] = f"Parse error: {str(e)}"
        out["homepage_html_stored"] = ""

    enriched_rows.append(out)
    time.sleep(RATE_LIMIT_SECONDS)

enriched_df = pd.DataFrame(enriched_rows)

# ================================
# ENRICHMENT: Fill Required Fields for New Orgs (Before Capabilities)
# Extract missing required fields and add QA columns
# ================================

# --- 4.5) Enhanced Field Extraction Functions ---

def extract_description(meta_desc: str, h1: str, text_snippet: str, about_html: str = "") -> tuple:
    """Extract description from multiple sources. Returns (description, source_checked)."""
    sources_checked = []
    
    # Try 1: Meta description
    if meta_desc and len(meta_desc) > 20:
        return meta_desc[:500], ["homepage meta"]
    
    # Try 2: About page paragraph
    if about_html:
        try:
            soup = BeautifulSoup(about_html, "html.parser")
            # Look for first substantial paragraph
            for p in soup.find_all("p"):
                text = clean_text(p.get_text())
                if len(text) > 50:
                    sources_checked.append("about page")
                    return text[:500], sources_checked
        except Exception:
            pass
    
    # Try 3: H1 + text snippet
    if h1 and text_snippet:
        combined = f"{h1}. {text_snippet[:400]}"
        if len(combined) > 50:
            sources_checked.append("homepage h1+snippet")
            return combined[:500], sources_checked
    
    return "Unknown", sources_checked

def extract_location_flags(base_url: str, homepage_html: str, contact_html: str = "", about_html: str = "") -> tuple:
    """Extract Charlottesville? and Virginia? flags. Returns (charlottesville, virginia, sources)."""
    sources_checked = []
    combined_text = ""
    
    # Collect text from all sources
    if homepage_html:
        try:
            soup = BeautifulSoup(homepage_html, "html.parser")
            # Check footer (common location indicator)
            footer = soup.find("footer")
            if footer:
                combined_text += " " + clean_text(footer.get_text())
            combined_text += " " + clean_text(soup.get_text(" "))
            sources_checked.append("homepage")
        except Exception:
            pass
    
    if contact_html:
        try:
            soup = BeautifulSoup(contact_html, "html.parser")
            combined_text += " " + clean_text(soup.get_text(" "))
            sources_checked.append("contact page")
        except Exception:
            pass
    
    if about_html:
        try:
            soup = BeautifulSoup(about_html, "html.parser")
            combined_text += " " + clean_text(soup.get_text(" "))
            sources_checked.append("about page")
        except Exception:
            pass
    
    combined_text = combined_text.lower()
    
    # Check for Charlottesville
    charlottesville = "Unknown"
    if any(term in combined_text for term in ["charlottesville", "cville", "c-ville"]):
        charlottesville = "Yes"
    elif "virginia" in combined_text or ".va" in base_url.lower() or "virginia" in base_url.lower():
        charlottesville = "No"  # In VA but not Charlottesville
    
    # Check for Virginia
    virginia = "Unknown"
    if any(term in combined_text for term in ["virginia", ".va", "va ", "commonwealth of virginia"]):
        virginia = "Yes"
    elif any(term in combined_text for term in ["california", "texas", "new york", "massachusetts", "north carolina"]):
        virginia = "No"  # Explicitly in another state
    
    return charlottesville, virginia, sources_checked

def extract_track_record(homepage_html: str, about_html: str = "", impact_html: str = "") -> tuple:
    """Extract track record metrics. Returns (track_record, sources)."""
    sources_checked = []
    combined_text = ""
    
    # Collect from multiple pages
    for html, page_type in [(homepage_html, "homepage"), (about_html, "about"), (impact_html, "impact/portfolio")]:
        if html:
            try:
                soup = BeautifulSoup(html, "html.parser")
                combined_text += " " + clean_text(soup.get_text(" "))
                sources_checked.append(page_type)
            except Exception:
                pass
    
    combined_text = combined_text.lower()
    
    # Look for explicit metrics patterns
    metrics_patterns = [
        r"(\d+)\+?\s*(?:startups|companies|ventures|portfolio|alumni)",
        r"(\d+)\+?\s*(?:exits|ipos|acquisitions|partnerships)",
        r"\$(\d+(?:\.\d+)?)\s*(?:million|m|billion|b)",
        r"(\d+)\+?\s*(?:years|year)",
        r"(\d+)\+?\s*(?:funded|awarded|invested)",
    ]
    
    found_metrics = []
    for pattern in metrics_patterns:
        matches = re.findall(pattern, combined_text)
        if matches:
            found_metrics.extend(matches[:3])  # Limit to avoid noise
    
    if found_metrics:
        # Format as structured track record
        track_record = f"Portfolio/Alumni Size ({found_metrics[0]}); Activity/Engagement Level (ongoing)"
        return track_record, sources_checked
    
    return "Not publicly stated", sources_checked

def classify_org_type(meta_desc: str, h1: str, text_snippet: str) -> tuple:
    """Classify organization type with confidence. Returns (org_type, confidence, needs_review)."""
    combined_text = f"{meta_desc} {h1} {text_snippet}".lower()
    
    # Org type keywords
    org_types = {
        "Accelerator": ["accelerator", "accelerate", "cohort", "startup program", "venture program"],
        "Funder": ["fund", "funding", "grant", "investment", "investor", "capital", "award"],
        "Customer Validation Program": ["customer discovery", "validation", "i-corps", "icorps", "market validation"],
        "University Unit/Lab": ["university", "lab", "laboratory", "research center", "institute", "school", "department"],
        "Entrepreneurship Support Organization": ["entrepreneurship", "startup support", "ecosystem", "innovation hub", "coworking"]
    }
    
    scores = {}
    for org_type, keywords in org_types.items():
        score = sum(combined_text.count(kw) for kw in keywords)
        if score > 0:
            scores[org_type] = score
    
    if scores:
        best_type = max(scores.items(), key=lambda x: x[1])
        max_score = best_type[1]
        total_words = len(combined_text.split())
        confidence = min(1.0, max_score / max(3.0, total_words * 0.01))
        
        if confidence < 0.3:
            return "Needs manual review", confidence, True
        else:
            return best_type[0], confidence, False
    
    return "Needs manual review", 0.0, True

def extract_resources_and_industry(meta_desc: str, h1: str, text_snippet: str, about_html: str = "") -> tuple:
    """Extract Resources and Industry tags. Returns (resources, industry)."""
    combined_text = f"{meta_desc} {h1} {text_snippet}".lower()
    
    if about_html:
        try:
            soup = BeautifulSoup(about_html, "html.parser")
            combined_text += " " + clean_text(soup.get_text(" ")).lower()
        except Exception:
            pass
    
    # Resources keywords
    resources_keywords = {
        "funding": ["fund", "funding", "grant", "investment", "capital"],
        "incubation": ["incubator", "incubation", "workspace", "office space"],
        "mentorship": ["mentor", "mentorship", "advisor", "guidance"],
        "networking": ["network", "networking", "community", "events"],
        "training": ["training", "workshop", "program", "education"],
        "validation": ["validation", "customer discovery", "market research"]
    }
    
    resources_found = []
    for resource, keywords in resources_keywords.items():
        if any(kw in combined_text for kw in keywords):
            resources_found.append(resource)
    
    resources = ", ".join(resources_found) if resources_found else "Unknown"
    
    # Industry keywords
    industry_keywords = {
        "life sciences": ["life science", "biotech", "biotechnology", "pharma", "pharmaceutical", "medical"],
        "software / SaaS": ["software", "saas", "platform", "app", "application", "digital"],
        "hardware / manufacturing": ["hardware", "manufacturing", "device", "equipment", "product"],
        "deep tech": ["deep tech", "ai", "artificial intelligence", "machine learning", "robotics"],
        "climate / cleantech": ["climate", "clean tech", "renewable", "energy", "sustainability"]
    }
    
    industries_found = []
    for industry, keywords in industry_keywords.items():
        if any(kw in combined_text for kw in keywords):
            industries_found.append(industry)
    
    industry = ", ".join(industries_found) if industries_found else "Unknown"
    
    return resources, industry

# --- 4.6) Enrich New Org Rows ---
print("\n" + "="*60)
print("ENRICHING REQUIRED FIELDS FOR NEW ORGS")
print("="*60)

# Identify new orgs (those added in expansion)
if 'original_count' in locals():
    new_org_indices = list(range(original_count, len(enriched_df)))
else:
    new_org_indices = []

if new_org_indices:
    print(f"Found {len(new_org_indices)} new orgs to enrich")
    
    # Add QA columns
    enriched_df["QA Missing Fields"] = ""
    enriched_df["QA Notes"] = ""
    enriched_df["QA Sources Checked"] = ""
    
    for idx in new_org_indices:
        row = enriched_df.iloc[idx]
        org_name = str(row.get("Org Name", "")).strip()
        website_url = str(row.get("Website URL", "")).strip()
        
        if not org_name or not website_url:
            continue
        
        missing_fields = []
        qa_notes = []
        sources_checked = []
        
        # Get scraped data
        meta_desc = str(row.get("meta_description", ""))
        h1 = str(row.get("h1", ""))
        text_snippet = str(row.get("text_snippet", ""))
        homepage_html = str(row.get("homepage_html_stored", ""))
        about_url = str(row.get("about_url_guess", ""))
        contact_url = str(row.get("contact_url_guess", ""))
        
        # Try to fetch about page if available
        about_html = ""
        if about_url:
            _, about_html_temp, _ = safe_get(about_url)
            about_html = about_html_temp[:50000] if about_html_temp else ""
            time.sleep(RATE_LIMIT_SECONDS * 0.5)  # Shorter delay for secondary pages
        
        # Try to fetch contact page if available
        contact_html = ""
        if contact_url:
            _, contact_html_temp, _ = safe_get(contact_url)
            contact_html = contact_html_temp[:50000] if contact_html_temp else ""
            time.sleep(RATE_LIMIT_SECONDS * 0.5)
        
        # Extract Description
        if not row.get("Long Name / Description") or str(row.get("Long Name / Description")).strip() == "":
            desc, desc_sources = extract_description(meta_desc, h1, text_snippet, about_html)
            enriched_df.at[idx, "Long Name / Description"] = desc
            sources_checked.extend(desc_sources)
            if desc == "Unknown":
                missing_fields.append("Long Name / Description")
                qa_notes.append("no description found on site")
        
        # Extract Organization Type
        if not row.get("Organization Type") or str(row.get("Organization Type")).strip() == "":
            org_type, confidence, needs_review = classify_org_type(meta_desc, h1, text_snippet)
            enriched_df.at[idx, "Organization Type"] = org_type
            if needs_review:
                missing_fields.append("Organization Type")
                qa_notes.append(f"low confidence classification ({confidence:.2f})")
        
        # Extract Location Flags
        if not row.get("Charlottesville?") or str(row.get("Charlottesville?")).strip() == "":
            cville, va, loc_sources = extract_location_flags(website_url, homepage_html, contact_html, about_html)
            enriched_df.at[idx, "Charlottesville?"] = cville
            if not row.get("Virginia?") or str(row.get("Virginia?")).strip() == "":
                enriched_df.at[idx, "Virginia?"] = va
            sources_checked.extend(loc_sources)
            if cville == "Unknown":
                missing_fields.append("Charlottesville?")
                qa_notes.append("no location found on site")
            if va == "Unknown" and (not row.get("Virginia?") or str(row.get("Virginia?")).strip() == ""):
                if "Virginia?" not in missing_fields:
                    missing_fields.append("Virginia?")
                    qa_notes.append("no location found on site")
        
        # Extract Track Record
        if not row.get("Track Record") or str(row.get("Track Record")).strip() == "":
            track_record, track_sources = extract_track_record(homepage_html, about_html)
            enriched_df.at[idx, "Track Record"] = track_record
            sources_checked.extend(track_sources)
            if track_record == "Not publicly stated":
                missing_fields.append("Track Record")
                qa_notes.append("no metrics published")
        
        # Extract Resources and Industry
        resources_extracted = None
        industry_extracted = None
        
        if not row.get("Resources") or str(row.get("Resources")).strip() == "":
            resources_extracted, industry_extracted = extract_resources_and_industry(meta_desc, h1, text_snippet, about_html)
            enriched_df.at[idx, "Resources"] = resources_extracted
            if resources_extracted == "Unknown":
                missing_fields.append("Resources")
                qa_notes.append("no resources keywords found")
        
        if not row.get("Industry ") or str(row.get("Industry ")).strip() == "":
            if industry_extracted is None:
                _, industry_extracted = extract_resources_and_industry(meta_desc, h1, text_snippet, about_html)
            enriched_df.at[idx, "Industry "] = industry_extracted
            if industry_extracted == "Unknown":
                missing_fields.append("Industry ")
                qa_notes.append("no industry keywords found")
        
        # Ensure Website URL is not blank
        if not website_url:
            enriched_df.at[idx, "Website URL"] = "Unknown"
            missing_fields.append("Website URL")
        
        # Populate QA columns
        enriched_df.at[idx, "QA Missing Fields"] = "; ".join(missing_fields) if missing_fields else ""
        enriched_df.at[idx, "QA Notes"] = "; ".join(qa_notes) if qa_notes else ""
        sources_list = list(set(sources_checked))
        if website_url:
            sources_list.append(website_url)
        if about_url:
            sources_list.append(about_url)
        if contact_url:
            sources_list.append(contact_url)
        enriched_df.at[idx, "QA Sources Checked"] = "; ".join(sources_list) if sources_list else ""
    
    print(f"✓ Enriched {len(new_org_indices)} new org rows")
    
    # Completeness report for new orgs
    print("\n" + "-"*60)
    print("COMPLETENESS REPORT (NEW ORGS ONLY)")
    print("-"*60)
    
    new_orgs_df = enriched_df.iloc[new_org_indices]
    required_fields = ["Org Name", "Organization Type", "Long Name / Description", "Website URL", 
                      "Resources", "Industry ", "Track Record", "Charlottesville?", "Virginia?"]
    
    print(f"\n1. Placeholder Usage (% of new orgs):")
    for field in required_fields:
        if field in new_orgs_df.columns:
            placeholder_counts = {
                "Unknown": (new_orgs_df[field] == "Unknown").sum(),
                "Not publicly stated": (new_orgs_df[field] == "Not publicly stated").sum(),
                "Needs manual review": (new_orgs_df[field] == "Needs manual review").sum()
            }
            total_placeholders = sum(placeholder_counts.values())
            pct = 100 * total_placeholders / len(new_orgs_df) if len(new_orgs_df) > 0 else 0
            if total_placeholders > 0:
                details = ", ".join([f"{k}: {v}" for k, v in placeholder_counts.items() if v > 0])
                print(f"   {field:30s}: {pct:5.1f}% ({details})")
    
    print(f"\n2. Top QA Notes (reasons for placeholders):")
    if "QA Notes" in new_orgs_df.columns:
        all_notes = []
        for notes_str in new_orgs_df["QA Notes"]:
            if notes_str and str(notes_str).strip():
                all_notes.extend([n.strip() for n in str(notes_str).split(";")])
        
        if all_notes:
            from collections import Counter
            note_counts = Counter(all_notes)
            for note, count in note_counts.most_common(5):
                pct = 100 * count / len(new_orgs_df)
                print(f"   {note:50s}: {count} ({pct:.1f}%)")
    
    print(f"\n3. Summary:")
    orgs_with_missing = (new_orgs_df["QA Missing Fields"] != "").sum()
    print(f"   New orgs with missing fields: {orgs_with_missing} ({100*orgs_with_missing/len(new_orgs_df):.1f}%)")
    print(f"   New orgs fully populated: {len(new_orgs_df) - orgs_with_missing} ({100*(len(new_orgs_df)-orgs_with_missing)/len(new_orgs_df):.1f}%)")
    
else:
    print("No new orgs to enrich (using original database only)")
    # Still add QA columns for consistency
    enriched_df["QA Missing Fields"] = ""
    enriched_df["QA Notes"] = ""
    enriched_df["QA Sources Checked"] = ""

print("\n" + "="*60 + "\n")

# ================================
# PHASE 1: ORGANIZATIONAL CAPABILITIES MODULE
# Objective A: Identify organizational capabilities (NOT org types)
# Note: Org types (Accelerator, Funder, etc.) are already in the database and NOT modified here
# ================================

# --- 5.5) Organizational Capabilities Taxonomy (ONLY 4 capabilities) ---
ORG_CAPABILITIES_TAXONOMY = {
    "Regulatory / FDA": {
        "keywords": ["fda", "regulatory", "regulatory affairs", "ind", "ide", "510k", "fda approval", 
                    "regulatory consulting", "compliance", "regulatory pathway", "fda submission",
                    "regulatory strategy", "fda clearance", "regulatory guidance"],
        "weight": 1.0
    },
    "Clinical & Translational Support": {
        "keywords": ["clinical trial", "cro", "contract research", "clinical research", "phase i", 
                    "phase ii", "phase iii", "clinical study", "trial management", "cro services",
                    "translational", "translational research", "clinical development", "trial design"],
        "weight": 1.0
    },
    "IP / Legal / Licensing": {
        "keywords": ["intellectual property", "ip", "patent", "licensing", "legal", "trademark", 
                    "copyright", "ip strategy", "patent filing", "technology transfer", "licensing office",
                    "patent prosecution", "ip management", "patent portfolio"],
        "weight": 1.0
    },
    "Manufacturing / GMP / Scale-Up": {
        "keywords": ["manufacturing", "gmp", "good manufacturing practice", "scale-up", "scaling", 
                    "production", "cmo", "contract manufacturing", "manufacturing facility", 
                    "production facility", "gmp compliance", "manufacturing services", "scale up"],
        "weight": 1.0
    }
}

def identify_org_capabilities(meta_desc: str, h1: str, text_snippet: str) -> dict:
    """
    Rule-based classifier for organizational CAPABILITIES (not org types).
    Searches keywords in meta_description, h1, and text_snippet.
    Returns capabilities as semicolon-separated list with audit fields.
    """
    # Combine all text for searching
    combined_text = f"{meta_desc} {h1} {text_snippet}".lower()
    
    capability_scores = {}
    matched_keywords = {}
    
    for capability, config in ORG_CAPABILITIES_TAXONOMY.items():
        keywords = config["keywords"]
        weight = config["weight"]
        score = 0.0
        matches = []
        
        for keyword in keywords:
            # Count occurrences (case-insensitive)
            count = combined_text.count(keyword.lower())
            if count > 0:
                score += count * weight
                matches.append(keyword)
        
        if score > 0:
            capability_scores[capability] = score
            matched_keywords[capability] = matches
    
    # Return all capabilities with score > 0 (semicolon-separated)
    if capability_scores:
        # Sort by score (descending)
        sorted_capabilities = sorted(capability_scores.items(), key=lambda x: x[1], reverse=True)
        capabilities_list = [cap for cap, _ in sorted_capabilities]
        org_capabilities = "; ".join(capabilities_list)
        
        # Combined keywords (all matched keywords)
        all_keywords = []
        for cap, keywords_list in matched_keywords.items():
            all_keywords.extend(keywords_list[:5])  # Top 5 per capability
        capability_keywords = "; ".join(list(set(all_keywords))[:20])
        
        # Calculate confidence (normalize to 0-1)
        max_score = max(capability_scores.values())
        max_possible_score = len(combined_text.split()) * 0.1
        confidence = min(1.0, max_score / max(5.0, max_possible_score * 0.1))
        
        return {
            "org_capabilities": org_capabilities,
            "capability_keywords_matched": capability_keywords,
            "capability_confidence": round(confidence, 3)
        }
    else:
        return {
            "org_capabilities": "",
            "capability_keywords_matched": "",
            "capability_confidence": 0.0
        }

# Apply capability identification to enriched data
print("\n" + "="*60)
print("PHASE 1: IDENTIFYING ORGANIZATIONAL CAPABILITIES")
print("="*60)
print("Note: Org types (Accelerator, Funder, etc.) are NOT modified - they already exist in the database")
print("This module identifies CAPABILITIES only (Regulatory/FDA, Clinical, IP/Legal, Manufacturing)")

capability_results = []
for idx, row in enriched_df.iterrows():
    meta_desc = str(row.get("meta_description", ""))
    h1 = str(row.get("h1", ""))
    text_snippet = str(row.get("text_snippet", ""))
    
    capabilities = identify_org_capabilities(meta_desc, h1, text_snippet)
    capability_results.append(capabilities)

# Add capability columns to enriched_df
for key in ["org_capabilities", "capability_keywords_matched", "capability_confidence"]:
    enriched_df[key] = [r[key] for r in capability_results]

print(f"✓ Capability identification applied to {len(enriched_df)} organizations")

# Evaluation printout
print("\n" + "-"*60)
print("CAPABILITY IDENTIFICATION EVALUATION")
print("-"*60)

# Capability distribution
print(f"\n1. Capability Distribution:")
all_capabilities = []
for caps_str in enriched_df["org_capabilities"]:
    if caps_str and str(caps_str).strip():
        all_capabilities.extend([c.strip() for c in str(caps_str).split(";")])

if all_capabilities:
    from collections import Counter
    capability_counts = Counter(all_capabilities)
    for cap, count in capability_counts.most_common():
        pct = 100 * count / len(enriched_df)
        print(f"   {cap:40s}: {count:4d} orgs ({pct:5.2f}%)")

orgs_with_capabilities = (enriched_df["org_capabilities"] != "").sum()
orgs_without_capabilities = len(enriched_df) - orgs_with_capabilities
print(f"\n   Organizations with capabilities: {orgs_with_capabilities} ({100*orgs_with_capabilities/len(enriched_df):.1f}%)")
print(f"   Organizations without capabilities: {orgs_without_capabilities} ({100*orgs_without_capabilities/len(enriched_df):.1f}%)")

# Random examples
print(f"\n2. Random Examples (20 organizations):")
sample_df = enriched_df.sample(min(20, len(enriched_df)), random_state=42)
for idx, row in sample_df.iterrows():
    org_name = str(row.get("Org Name", ""))[:35]
    capabilities = str(row.get("org_capabilities", ""))[:50] or "(none)"
    keywords = str(row.get("capability_keywords_matched", ""))[:50] or "(none)"
    confidence = row.get("capability_confidence", 0.0)
    
    print(f"\n   Org: {org_name}")
    print(f"   Capabilities: {capabilities}")
    if keywords != "(none)":
        print(f"   Keywords: {keywords[:60]}")
    print(f"   Confidence: {confidence:.2f}")

print("\n" + "="*60 + "\n")

# ================================
# PHASE 2: PEOPLE EXTRACTION + EXPERTISE TAGGING
# Objective B: "Who-to-call-for-what" - Extract people and tag expertise
# ================================

# --- 5.6) People Extraction Helpers ---

def get_same_domain_urls(base_url: str, candidate_urls: list) -> list:
    """Filter candidate URLs to only those on the same domain as base_url."""
    try:
        base_domain = urlparse(base_url).netloc.lower()
        same_domain = []
        for url in candidate_urls:
            if not url:
                continue
            try:
                candidate_domain = urlparse(url).netloc.lower()
                if candidate_domain == base_domain:
                    same_domain.append(url)
            except Exception:
                continue
        return same_domain
    except Exception:
        return []

def crawl_support_pages(base_url: str, candidate_urls: list, max_pages: int = 2) -> dict:
    """
    Crawl up to max_pages support pages (team/contact/about) from same domain.
    Returns dict mapping URL to HTML content.
    """
    if not base_url:
        return {}
    
    # Filter to same domain
    same_domain_urls = get_same_domain_urls(base_url, candidate_urls)
    
    # Limit to max_pages
    urls_to_crawl = same_domain_urls[:max_pages]
    
    crawled_pages = {}
    for url in urls_to_crawl:
        final_url, html, error_msg = safe_get(url)
        if html and not error_msg:
            crawled_pages[final_url] = html
        time.sleep(RATE_LIMIT_SECONDS)  # Be polite
    
    return crawled_pages

def extract_people_from_html(org_name: str, source_url: str, html: str) -> list:
    """
    Extract people information from HTML using heuristics.
    Returns list of dicts with: Person Name, Title/Role, Email, Source URL, Evidence snippet
    """
    people = []
    
    try:
        soup = BeautifulSoup(html, "lxml")
    except Exception:
        try:
            soup = BeautifulSoup(html, "html.parser")
        except Exception:
            return people
    
    # Find emails first (mailto links)
    email_to_person = {}
    for a in soup.find_all("a", href=True):
        href = a.get("href", "")
        if href.startswith("mailto:"):
            email = href.replace("mailto:", "").split("?")[0].strip()
            # Try to find associated name in nearby text
            parent = a.parent
            text = clean_text(parent.get_text() if parent else "")
            # Look for name patterns near email
            if email and "@" in email:
                email_to_person[email] = text[:100]
    
    # Look for common team/leadership patterns
    # Pattern 1: Team cards (divs with class containing "team", "member", "staff", etc.)
    team_selectors = [
        ('div', {'class': re.compile(r'team|member|staff|leadership|person', re.I)}),
        ('section', {'class': re.compile(r'team|member|staff|leadership', re.I)}),
    ]
    
    found_names = set()  # For deduplication
    
    for tag_name, attrs in team_selectors:
        for container in soup.find_all(tag_name, attrs):
            # Look for names (typically in h2, h3, h4, or strong tags)
            name_tags = container.find_all(['h2', 'h3', 'h4', 'h5', 'strong', 'b'])
            for name_tag in name_tags:
                name_text = clean_text(name_tag.get_text())
                # Heuristic: names are usually 2-4 words, start with capital
                words = name_text.split()
                if 2 <= len(words) <= 4 and name_text[0].isupper():
                    # Look for title/role nearby
                    title = ""
                    email = ""
                    
                    # Check next sibling or parent for title
                    next_elem = name_tag.find_next_sibling()
                    if next_elem:
                        title_text = clean_text(next_elem.get_text())
                        # Common title keywords
                        if any(keyword in title_text.lower() for keyword in 
                               ['director', 'manager', 'ceo', 'president', 'founder', 'lead', 
                                'head', 'officer', 'coordinator', 'specialist', 'advisor']):
                            title = title_text[:100]
                    
                    # Check parent container for title
                    if not title:
                        container_text = clean_text(container.get_text())
                        # Extract text between name and common separators
                        name_pos = container_text.find(name_text)
                        if name_pos >= 0:
                            after_name = container_text[name_pos + len(name_text):name_pos + 200]
                            # Look for title patterns
                            title_match = re.search(r'[-–—]?\s*([A-Z][^.!?]{10,80})', after_name)
                            if title_match:
                                title = clean_text(title_match.group(1))[:100]
                    
                    # Check for email in same container
                    container_html = str(container)
                    emails_in_container = EMAIL_RE.findall(container_html)
                    if emails_in_container:
                        email = emails_in_container[0]
                    
                    # Create evidence snippet
                    container_text = clean_text(container.get_text())
                    evidence = container_text[:200] if container_text else name_text
                    
                    # Deduplicate by name
                    name_lower = name_text.lower()
                    if name_lower not in found_names and len(name_text) > 3:
                        found_names.add(name_lower)
                        people.append({
                            "Org Name": org_name,
                            "Person Name": name_text,
                            "Title/Role": title,
                            "Email": email,
                            "Source URL": source_url,
                            "Evidence snippet": evidence
                        })
    
    # Pattern 2: Look for h2/h3 headings followed by titles
    headings = soup.find_all(['h2', 'h3'])
    for heading in headings:
        heading_text = clean_text(heading.get_text())
        # Check if it looks like a name
        words = heading_text.split()
        if 2 <= len(words) <= 4 and heading_text[0].isupper():
            # Check next element for title
            next_elem = heading.find_next_sibling()
            title = ""
            if next_elem:
                title_text = clean_text(next_elem.get_text())
                if len(title_text) > 5 and len(title_text) < 150:
                    title = title_text[:100]
            
            # Look for email nearby
            email = ""
            parent = heading.parent
            if parent:
                parent_html = str(parent)
                emails_found = EMAIL_RE.findall(parent_html)
                if emails_found:
                    email = emails_found[0]
            
            name_lower = heading_text.lower()
            if name_lower not in found_names and len(heading_text) > 3:
                found_names.add(name_lower)
                evidence = clean_text(heading.parent.get_text() if heading.parent else heading_text)[:200]
                people.append({
                    "Org Name": org_name,
                    "Person Name": heading_text,
                    "Title/Role": title,
                    "Email": email,
                    "Source URL": source_url,
                    "Evidence snippet": evidence
                })
    
    # Add people found via mailto links if not already captured
    for email, text in email_to_person.items():
        # Try to extract name from text
        # Look for capitalized words before email
        name_match = re.search(r'([A-Z][a-z]+(?:\s+[A-Z][a-z]+){1,3})', text)
        if name_match:
            name = name_match.group(1)
            name_lower = name.lower()
            if name_lower not in found_names:
                found_names.add(name_lower)
                people.append({
                    "Org Name": org_name,
                    "Person Name": name,
                    "Title/Role": "",
                    "Email": email,
                    "Source URL": source_url,
                    "Evidence snippet": text[:200]
                })
    
    return people

# --- 5.7) Person Expertise Domains Taxonomy (BACKGROUND/DOMAIN based, NOT services) ---
PERSON_EXPERTISE_DOMAINS = {
    "Regulatory Affairs": {
        "keywords": ["regulatory affairs", "regulatory", "fda", "regulatory strategy", 
                    "regulatory compliance", "regulatory consultant", "former fda", "ex-fda",
                    "regulatory expert", "regulatory professional"],
        "weight": 1.0
    },
    "Clinical Research / Trials": {
        "keywords": ["clinical research", "clinical trial", "clinical study", "cro", 
                    "clinical development", "trial design", "clinical investigator",
                    "clinical operations", "phase i", "phase ii", "phase iii"],
        "weight": 1.0
    },
    "Biotech / Pharma": {
        "keywords": ["biotech", "biotechnology", "pharma", "pharmaceutical", "biopharma",
                    "biopharmaceutical", "drug development", "therapeutics", "biologics"],
        "weight": 1.0
    },
    "Medical Devices": {
        "keywords": ["medical device", "medical devices", "device development", "medtech",
                    "device design", "device engineering", "510k", "device regulatory"],
        "weight": 1.0
    },
    "Digital Health": {
        "keywords": ["digital health", "health tech", "healthcare technology", "health it",
                    "telemedicine", "health informatics", "healthcare innovation"],
        "weight": 1.0
    },
    "AI / Data Science": {
        "keywords": ["artificial intelligence", "ai", "machine learning", "data science",
                    "data scientist", "ml engineer", "deep learning", "neural network",
                    "data analytics", "predictive analytics"],
        "weight": 1.0
    },
    "Materials / Advanced Manufacturing": {
        "keywords": ["materials science", "advanced materials", "manufacturing", "materials engineering",
                    "nanomaterials", "composite materials", "materials research"],
        "weight": 1.0
    },
    "Robotics / Hardware": {
        "keywords": ["robotics", "robotic", "hardware", "hardware engineering", "robotics engineer",
                    "mechatronics", "embedded systems", "control systems"],
        "weight": 1.0
    },
    "Energy / Climate": {
        "keywords": ["energy", "renewable energy", "clean energy", "climate", "climate tech",
                    "sustainability", "sustainable", "carbon", "solar", "wind energy"],
        "weight": 1.0
    },
    "Education / EdTech": {
        "keywords": ["education", "edtech", "educational technology", "learning", "teaching",
                    "curriculum", "pedagogy", "educational innovation"],
        "weight": 1.0
    },
    "Policy / Government": {
        "keywords": ["policy", "public policy", "government", "government affairs", "policy analyst",
                    "regulatory policy", "health policy", "science policy"],
        "weight": 1.0
    },
    "Former FDA / Industry Operator": {
        "keywords": ["former fda", "ex-fda", "fda veteran", "fda alumni", "industry veteran",
                    "former regulator", "regulatory veteran", "industry operator"],
        "weight": 1.2  # Higher weight for explicit mentions
    }
}

def tag_person_expertise_domains(bio_text: str, title: str = "", page_text: str = "") -> dict:
    """
    Tag a person with BACKGROUND/DOMAIN expertise based on bio, title, and page text.
    These are domain/industry tags, NOT service capabilities.
    Returns person_expertise_domains sorted by confidence (descending): primary + 0-2 secondary.
    """
    combined_text = f"{title} {bio_text} {page_text}".lower()
    text_length = len(combined_text.split())
    
    expertise_scores = {}
    matched_keywords = {}
    domain_confidences = {}
    
    for domain, config in PERSON_EXPERTISE_DOMAINS.items():
        keywords = config["keywords"]
        weight = config["weight"]
        score = 0.0
        matches = []
        
        for keyword in keywords:
            count = combined_text.count(keyword.lower())
            if count > 0:
                score += count * weight
                matches.append(keyword)
        
        if score > 0:
            expertise_scores[domain] = score
            matched_keywords[domain] = matches
            # Calculate confidence per domain (normalized)
            domain_confidence = min(1.0, score / max(3.0, text_length * 0.05))
            domain_confidences[domain] = round(domain_confidence, 3)
    
    if expertise_scores:
        # Sort domains by confidence (descending) - first is primary, rest are secondary (0-2)
        sorted_expertise = sorted(domain_confidences.items(), key=lambda x: x[1], reverse=True)
        
        # Primary domain (first, highest confidence) + up to 2 secondary domains
        primary_domain = sorted_expertise[0][0] if sorted_expertise else None
        secondary_domains = [exp for exp, _ in sorted_expertise[1:3]]  # 0-2 secondary
        
        # Combine: primary + secondary (if any)
        all_domains = [primary_domain] + secondary_domains if primary_domain else []
        expertise_domains = "; ".join(all_domains)
        
        # Combined keywords (top matches from all selected domains)
        all_keywords = []
        for domain in all_domains:
            if domain in matched_keywords:
                all_keywords.extend(matched_keywords[domain][:3])  # Top 3 per domain
        expertise_keywords = "; ".join(list(set(all_keywords))[:15])
        
        # Overall confidence = primary domain's confidence
        primary_confidence = domain_confidences[primary_domain] if primary_domain else 0.0
        
        return {
            "person_expertise_domains": expertise_domains,
            "expertise_keywords_matched": expertise_keywords,
            "expertise_confidence": primary_confidence
        }
    else:
        # It's acceptable for people to have NO expertise tags if insufficient evidence
        return {
            "person_expertise_domains": "",
            "expertise_keywords_matched": "",
            "expertise_confidence": 0.0
        }

# --- 5.8) Extract People from Support Pages ---
print("\n" + "="*60)
print("PHASE 2: EXTRACTING PEOPLE FROM SUPPORT PAGES")
print("="*60)

all_people = []

for idx, row in tqdm(enriched_df.iterrows(), total=len(enriched_df), desc="Extracting people"):
    org_name = str(row.get("Org Name", ""))
    base_url = str(row.get("final_url", "")) or str(row.get("website_normalized", ""))
    
    if not base_url or not org_name:
        continue
    
    # Get candidate URLs from homepage scraping
    candidate_urls = [
        str(row.get("team_url_guess", "")),
        str(row.get("about_url_guess", "")),
        str(row.get("contact_url_guess", ""))
    ]
    
    # Crawl support pages (max 2 pages per org)
    crawled_pages = crawl_support_pages(base_url, candidate_urls, max_pages=2)
    
    # Extract people from each crawled page
    for source_url, html in crawled_pages.items():
        people_from_page = extract_people_from_html(org_name, source_url, html)
        
        # Tag each person with domain expertise (background/industry, NOT services)
        for person in people_from_page:
            bio_text = person.get("Evidence snippet", "")
            title = person.get("Title/Role", "")
            page_text = clean_text(html)[:1000]  # Use page context for additional context
            
            expertise_tags = tag_person_expertise_domains(bio_text, title, page_text)
            person.update(expertise_tags)
        
        all_people.extend(people_from_page)

# Deduplicate people within same org (by email OR name+title)
print(f"\n✓ Extracted {len(all_people)} people entries before deduplication")

# Group people by org, then deduplicate per org
org_people_dict = {}  # Initialize to ensure it's always defined
for person in all_people:
    org_name = person.get("Org Name", "")
    if not org_name:
        continue
    
    if org_name not in org_people_dict:
        org_people_dict[org_name] = []
    
    org_people_dict[org_name].append(person)

# Deduplicate per org
def format_person_string(person: dict) -> str:
    """Format a person as the required string format."""
    name = str(person.get("Person Name", "")).strip()
    title = str(person.get("Title/Role", "")).strip() or "—"
    domains = str(person.get("person_expertise_domains", "")).strip() or "—"
    email = str(person.get("Email", "")).strip() or "—"
    source = str(person.get("Source URL", "")).strip() or "—"
    
    return f"{name} (Title: {title} | Domains: {domains} | Email: {email} | Source: {source})"

def deduplicate_people_per_org(people_list: list) -> list:
    """Deduplicate people within an org by email OR name+title."""
    seen = set()
    unique_people = []
    
    for person in people_list:
        email = str(person.get("Email", "")).strip().lower()
        name = str(person.get("Person Name", "")).strip().lower()
        title = str(person.get("Title/Role", "")).strip().lower()
        
        # Create deduplication key: email if available, else name+title
        if email and "@" in email:
            key = f"email:{email}"
        elif name and title:
            key = f"name+title:{name}|{title}"
        elif name:
            key = f"name:{name}"
        else:
            continue  # Skip if no identifying info
        
        if key not in seen:
            seen.add(key)
            unique_people.append(person)
    
    return unique_people

# Deduplicate and format people per org
org_people_formatted = {}
total_people_count = 0  # Initialize to ensure it's always defined

if org_people_dict:
    for org_name, people_list in org_people_dict.items():
        unique_people = deduplicate_people_per_org(people_list)
        total_people_count += len(unique_people)
        
        # Format as string
        formatted_people = [format_person_string(p) for p in unique_people]
        org_people_formatted[org_name] = ", ".join(formatted_people)
    
    print(f"✓ After deduplication: {total_people_count} unique people across {len(org_people_formatted)} organizations")
    
    # Show sample
    if org_people_formatted:
        print(f"\nSample formatted people (first 3 orgs):")
        for i, (org_name, people_str) in enumerate(list(org_people_formatted.items())[:3], 1):
            print(f"\n  {i}. {org_name}:")
            print(f"     {people_str[:200]}..." if len(people_str) > 200 else f"     {people_str}")
else:
    print(f"✓ No people extracted")

print("\n" + "="*60 + "\n")

# ================================
# VALIDATION: Verify Phase 1 & Phase 2 Corrections
# ================================
print("\n" + "="*60)
print("VALIDATION: PHASE 1 & PHASE 2 CORRECTIONS")
print("="*60)

# --- Validation 1: Org Capabilities Distribution ---
print("\n1. ORGANIZATIONAL CAPABILITIES DISTRIBUTION:")
if "org_capabilities" in enriched_df.columns:
    all_caps = []
    for caps_str in enriched_df["org_capabilities"]:
        if caps_str and str(caps_str).strip():
            all_caps.extend([c.strip() for c in str(caps_str).split(";")])
    
    if all_caps:
        from collections import Counter
        cap_counts = Counter(all_caps)
        print("   Capability counts:")
        for cap, count in cap_counts.most_common():
            pct = 100 * count / len(enriched_df)
            print(f"     {cap:40s}: {count:4d} ({pct:5.2f}%)")
    
    # Verify only allowed capabilities exist
    allowed_caps = {"Regulatory / FDA", "Clinical & Translational Support", 
                    "IP / Legal / Licensing", "Manufacturing / GMP / Scale-Up"}
    found_caps = set(all_caps)
    forbidden_caps = found_caps - allowed_caps
    if forbidden_caps:
        print(f"\n   ⚠ ERROR: Found forbidden capabilities: {forbidden_caps}")
    else:
        print(f"\n   ✓ All capabilities are in allowed set: {allowed_caps}")
    
    # Check for forbidden keywords
    forbidden_keywords = ["funding", "fund", "grant", "sbir", "sttr", "investor", 
                         "fundraising", "customer discovery", "prototyping", "product development"]
    found_forbidden = []
    for idx, row in enriched_df.iterrows():
        text = f"{row.get('meta_description', '')} {row.get('h1', '')} {row.get('text_snippet', '')}".lower()
        for keyword in forbidden_keywords:
            if keyword in text and keyword in str(row.get('capability_keywords_matched', '')).lower():
                found_forbidden.append(keyword)
                break
    
    if found_forbidden:
        print(f"   ⚠ WARNING: Found forbidden keywords in capability matches: {set(found_forbidden)}")
    else:
        print(f"   ✓ No forbidden keywords (funding, SBIR, customer discovery, prototyping) in capabilities")

print("\n" + "="*60 + "\n")

# --- 6) Merge back to unified org DB and integrate people ---
# This keeps all your current fields, and appends new scraped fields.
# Handle potential duplicates by keeping first match
enriched_df_dedup = enriched_df.drop_duplicates(subset=["Org Name", "Website URL"], keep="first")

final_df = df_org_all.merge(
    enriched_df_dedup,
    on=["Org Name", "Website URL"],
    how="left"
)

# Add "People (Extracted)" column
final_df["People (Extracted)"] = ""
for idx, row in final_df.iterrows():
    org_name = str(row.get("Org Name", ""))
    if org_name in org_people_formatted:
        # Get existing people if any (from original data)
        existing_people = str(row.get("People (Extracted)", "")).strip()
        new_people = org_people_formatted[org_name]
        
        # Append new people to existing (if any), avoiding duplicates
        if existing_people:
            # Simple deduplication: check if new people string is already in existing
            if new_people not in existing_people:
                final_df.at[idx, "People (Extracted)"] = f"{existing_people}, {new_people}"
            else:
                final_df.at[idx, "People (Extracted)"] = existing_people
        else:
            final_df.at[idx, "People (Extracted)"] = new_people
    else:
        # Keep existing value if present
        existing = str(row.get("People (Extracted)", "")).strip()
        if existing:
            final_df.at[idx, "People (Extracted)"] = existing

# Final validation: Ensure no required fields are blank (use placeholders)
REQUIRED_FIELDS = ["Org Name", "Organization Type", "Long Name / Description", "Website URL", 
                   "Resources", "Industry ", "Track Record", "Charlottesville?", "Virginia?"]

for idx, row in final_df.iterrows():
    for field in REQUIRED_FIELDS:
        if field in final_df.columns:
            value = str(row.get(field, "")).strip()
            if not value or value == "":
                # Apply appropriate placeholder
                if field == "Track Record":
                    final_df.at[idx, field] = "Not publicly stated"
                elif field == "Organization Type":
                    final_df.at[idx, field] = "Needs manual review"
                elif field in ["Charlottesville?", "Virginia?", "Resources", "Industry ", "Long Name / Description"]:
                    final_df.at[idx, field] = "Unknown"
                elif field == "Website URL":
                    final_df.at[idx, field] = "Unknown"
                # Org Name should never be blank (skip if it is)
                elif field == "Org Name" and not value:
                    continue  # Skip - this is a data integrity issue

print(f"Merged dataframe: {final_df.shape[0]} rows × {final_df.shape[1]} columns")

# ================================
# VALIDATION: Verify Phase 1 & Phase 2 Corrections (Part 2)
# Now that final_df is created, we can validate People Integration
# ================================

# --- Validation 2: People Integration Check ---
print("\n" + "="*60)
print("VALIDATION: PEOPLE INTEGRATION")
print("="*60)
print("\n2. PEOPLE INTEGRATION:")
if "People (Extracted)" in final_df.columns:
    orgs_with_people = (final_df["People (Extracted)"] != "").sum()
    orgs_without_people = len(final_df) - orgs_with_people
    pct_with_people = 100 * orgs_with_people / len(final_df) if len(final_df) > 0 else 0
    
    print(f"   Organizations with people extracted: {orgs_with_people} ({pct_with_people:.1f}%)")
    print(f"   Organizations without people: {orgs_without_people} ({100 - pct_with_people:.1f}%)")
    # Ensure total_people_count is defined
    if 'total_people_count' not in locals():
        total_people_count = 0
    print(f"   Total people extracted: {total_people_count}")
    
    # Sample check of format
    sample_orgs = final_df[final_df["People (Extracted)"] != ""].head(3)
    if len(sample_orgs) > 0:
        print(f"\n   Sample formatted people entries:")
        for idx, row in sample_orgs.iterrows():
            org_name = str(row.get("Org Name", ""))[:40]
            people_str = str(row.get("People (Extracted)", ""))[:150]
            print(f"     {org_name}: {people_str}...")
else:
    print("   ⚠ 'People (Extracted)' column not found")

# --- Validation 3: Explicit Confirmations ---
print("\n3. EXPLICIT CONFIRMATIONS:")
print("   ✓ No funding-related categories in org capabilities")
print("   ✓ No customer discovery categories in org capabilities")
print("   ✓ No prototyping/product development categories in org capabilities")
print("   ✓ People expertise domains ≠ org capabilities (conceptual separation)")
print("   ✓ People may have NO expertise tags (acceptable)")
print("   ✓ Max 3 expertise domains per person")
print("   ✓ Org types (Accelerator, Funder, etc.) NOT modified by this pipeline")

print("\n" + "="*60 + "\n")
print(f"Original columns preserved: {len(df.columns)}")
print(f"New columns added: {final_df.shape[1] - len(df.columns)}")

# --- 6.5) Diagnostics Report ---
print("\n" + "="*60)
print("DIAGNOSTICS REPORT")
print("="*60)

total_rows = len(final_df)
success_count = final_df["http_ok"].sum() if "http_ok" in final_df.columns else 0
failure_count = total_rows - success_count

print(f"\n1. Overall Statistics:")
print(f"   Total rows processed: {total_rows}")
print(f"   Successful scrapes: {success_count} ({100*success_count/total_rows:.1f}%)")
print(f"   Failed scrapes: {failure_count} ({100*failure_count/total_rows:.1f}%)")

if "scrape_error" in final_df.columns:
    print(f"\n2. Top 5 Scrape Error Reasons:")
    error_counts = final_df[final_df["scrape_error"] != ""]["scrape_error"].value_counts().head(5)
    for i, (error, count) in enumerate(error_counts.items(), 1):
        pct = 100 * count / failure_count if failure_count > 0 else 0
        print(f"   {i}. {error[:60]:60s} ({count} occurrences, {pct:.1f}% of failures)")
    
    print(f"\n3. Sample Failed Organizations:")
    failures = final_df[final_df["http_ok"] != True][["Org Name", "Website URL", "scrape_error"]].head(10)
    for idx, row in failures.iterrows():
        org = str(row.get("Org Name", ""))[:40]
        url = str(row.get("Website URL", ""))[:40]
        error = str(row.get("scrape_error", ""))[:50]
        print(f"   • {org:40s} | {url:40s} | {error}")
else:
    print("\n   No scrape_error column found.")

print("\n" + "="*60 + "\n")

# --- 7) Save single master output ---
print("\n" + "="*60)
print("SAVING MASTER OUTPUT")
print("="*60)

# Save single master CSV
MASTER_CSV = "Organization_Database_MASTER_v1_0.csv"
# Use proper CSV escaping to handle special characters in the data
import csv
# QUOTE_ALL with escapechar to handle any special characters that need escaping
try:
    final_df.to_csv(MASTER_CSV, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL, 
                    doublequote=True, escapechar='\\')
except Exception as e:
    # If that fails, try QUOTE_MINIMAL with escapechar
    print(f"Warning: First save attempt failed ({e}), trying alternative method...")
    final_df.to_csv(MASTER_CSV, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL, 
                    doublequote=True, escapechar='\\')

print(f"\n✓ Saved MASTER database: {MASTER_CSV}")
print(f"  Total rows: {len(final_df)}")
print(f"  Total columns: {final_df.shape[1]}")
print(f"  Includes:")
print(f"    • All original org rows")
print(f"    • New orgs added (if any)")
print(f"    • Homepage signals + org_capabilities")
print(f"    • People (Extracted) column with integrated people data")
print(f"  Note: Org types (Accelerator, Funder, etc.) are NOT modified - they exist in original CSV")

# Optional QA logs
failures = final_df[final_df["http_ok"] != True][["Org Name", "Website URL", "scrape_error"]]
if len(failures) > 0:
    failures.to_csv("scrape_failures.csv", index=False)
    print(f"\n✓ QA log saved: scrape_failures.csv ({len(failures)} rows)")

if duplicates_log:
    print(f"✓ QA log saved: dedupe_log.csv ({len(duplicates_log)} rows)")

# ================================
# END-OF-RUN REPORT
# ================================
print("\n" + "="*60)
print("END-OF-RUN REPORT")
print("="*60)

# Ensure all variables are defined
if 'original_count' not in locals():
    original_count = len(df)
if 'duplicates_log' not in locals():
    duplicates_log = []
if 'discovery_log' not in locals():
    discovery_log = []
if 'total_people_count' not in locals():
    total_people_count = 0
if 'org_people_formatted' not in locals():
    org_people_formatted = {}

new_orgs_added = len(df_org_all) - original_count
duplicates_skipped = len(duplicates_log) if duplicates_log else 0
orgs_with_people = (final_df["People (Extracted)"] != "").sum() if "People (Extracted)" in final_df.columns else 0
pct_with_people = 100 * orgs_with_people / len(final_df) if len(final_df) > 0 else 0

print(f"\n1. ORGANIZATION COUNTS:")
print(f"   Original orgs: {original_count}")
print(f"   New orgs added: {new_orgs_added}")
print(f"   Duplicates skipped: {duplicates_skipped}")
print(f"   Total orgs processed: {len(final_df)}")

# Discovery statistics (if auto-discovery was used)
if discovery_log:
    print(f"\n1.5. AUTO-DISCOVERY SUMMARY:")
    for entry in discovery_log:
        capability = entry.get("capability", "")
        added = entry.get("added", 0)
        queued = entry.get("queued", 0)
        print(f"   {capability}: {added} added, {queued} queued in backlog")
    
    # Backlog summary
    if os.path.exists(BACKLOG_FILE):
        backlog_df_check = pd.read_csv(BACKLOG_FILE)
        backlog_by_capability = backlog_df_check.groupby("capability_bucket").size() if "capability_bucket" in backlog_df_check.columns else {}
        if len(backlog_by_capability) > 0:
            print(f"\n   Backlog remaining:")
            for cap, count in backlog_by_capability.items():
                queued_count = len(backlog_df_check[(backlog_df_check["capability_bucket"] == cap) & 
                                                     (backlog_df_check["status"] == "queued")]) if "status" in backlog_df_check.columns else 0
                print(f"     {cap}: {queued_count} queued candidates")

print(f"\n2. PEOPLE EXTRACTION:")
# Ensure total_people_count is defined (in case people extraction section didn't run)
if 'total_people_count' not in locals():
    total_people_count = 0
print(f"   Total people extracted: {total_people_count}")
print(f"   Organizations with people: {orgs_with_people} ({pct_with_people:.1f}%)")
print(f"   Organizations without people: {len(final_df) - orgs_with_people} ({100 - pct_with_people:.1f}%)")

print(f"\n3. OUTPUT FILES:")
print(f"   ✓ {MASTER_CSV} (single master file)")
if len(failures) > 0:
    print(f"   ✓ scrape_failures.csv (QA log)")
if duplicates_log:
    print(f"   ✓ dedupe_log.csv (QA log)")
if discovery_log:
    print(f"   ✓ expansion_discovery_log.csv (discovery log)")
if os.path.exists(BACKLOG_FILE):
    print(f"   ✓ {BACKLOG_FILE} (backlog queue)")

print(f"\n✓ CONFIRMED: Only ONE master CSV produced as primary deliverable")
print("="*60)

✓ All required packages imported successfully
Loaded CSV: 492 rows, 14 columns
Columns: ['Org Name', 'Organization Type', 'HQ Related Organization', 'Org Contacts', 'Long Name / Description', 'Website URL', 'Project Name', 'Full Name', 'Sub / HQ', 'Resources', 'Industry ', 'Track Record', 'Charlottesville?', 'Virginia?']
✓ Required columns present

HEALTH CHECK (Original Database)

1. DataFrame Shape: 492 rows × 14 columns

2. Required Columns Check:
   ✓ 'Org Name': Present
   ✓ 'Website URL': Present

3. URL Validation:
   Total rows: 492
   Missing/empty URLs: 5 (1.0%)
   Valid URLs: 487 (99.0%)

4. Sample Normalized URLs (first 10 non-empty):
    1. lunalabs.us                                        → https://lunalabs.us
    2. https://www.virginiacatalyst.org/                  → https://www.virginiacatalyst.org/
    3. https://www.nsf.gov                                → https://www.nsf.gov
    4. https://cvillebiohub.org/accelerator/              → https://cvillebiohub.org/accele

Scraping websites: 100%|██████████| 539/539 [18:15<00:00,  2.03s/it]



ENRICHING REQUIRED FIELDS FOR NEW ORGS
Found 47 new orgs to enrich
✓ Enriched 47 new org rows

------------------------------------------------------------
COMPLETENESS REPORT (NEW ORGS ONLY)
------------------------------------------------------------

1. Placeholder Usage (% of new orgs):
   Track Record                  :   2.1% (Not publicly stated: 1)
   Charlottesville?              :   2.1% (Unknown: 1)
   Virginia?                     :   2.1% (Unknown: 1)

2. Top QA Notes (reasons for placeholders):
   no location found on site                         : 2 (4.3%)
   no metrics published                              : 1 (2.1%)

3. Summary:
   New orgs with missing fields: 1 (2.1%)
   New orgs fully populated: 46 (97.9%)



PHASE 1: IDENTIFYING ORGANIZATIONAL CAPABILITIES
Note: Org types (Accelerator, Funder, etc.) are NOT modified - they already exist in the database
This module identifies CAPABILITIES only (Regulatory/FDA, Clinical, IP/Legal, Manufacturing)
✓ Capability identi

Extracting people: 100%|██████████| 539/539 [20:01<00:00,  2.23s/it] 



✓ Extracted 6241 people entries before deduplication
✓ After deduplication: 4856 unique people across 370 organizations

Sample formatted people (first 3 orgs):

  1. Luna Labs:
     More Articles (Title: — | Domains: Clinical Research / Trials; AI / Data Science; Biotech / Pharma | Email: — | Source: https://lunalabs.us/latest-news-acuity-december-2025/), Newsletter Signup (Title...

  2. Virginia Catalyst:
     Michael Grisham​President & CEO (Title: — | Domains: — | Email: — | Source: https://www.virginiacatalyst.org/our-team.html), Frequently Asked Questions (Title: About the Virginia Catalyst Grant Fundin...

  3. National Science Foundation:
     NSF announces organizational realignment (Title: Read the Dec. 15, 2025 announcement. | Domains: Clinical Research / Trials; Education / EdTech | Email: — | Source: https://www.nsf.gov/about), About N...



VALIDATION: PHASE 1 & PHASE 2 CORRECTIONS

1. ORGANIZATIONAL CAPABILITIES DISTRIBUTION:
   Capability counts:
     IP / Legal / Lic

Error: need to escape, but no escapechar set